# Models to use
codewithdark/vit-chest-xray
VIT - Transformer based
https://arxiv.org/pdf/2010.11929
While the Transformer architecture has become the de-facto standard for natural
language processing tasks, its applications to computer vision remain limited. In
vision, attention is either applied in conjunction with convolutional networks, or
used to replace certain components of convolutional networks while keeping their
overall structure in place. We show that this reliance on CNNs is not necessary
and a pure transformer applied directly to sequences of image patches can perform
very well on image classification tasks. When pre-trained on large amounts of
data and transferred to multiple mid-sized or small image recognition benchmarks
(ImageNet, CIFAR-100, VTAB, etc.), Vision Transformer (ViT) attains excellent
results compared to state-of-the-art convolutional networks while requiring substantially fewer computational resources to train.

DataMinds/cnn_brain_tumor_classification 
CNN
Trained on brain tumor data that used MRIs. 

dyaminda/alexnet-pneumonia
Alexnet - ImageNet Subclass?
https://proceedings.neurips.cc/paper_files/paper/2012/file/c399862d3b9d6b76c8436e924a68c45b-Paper.pdf
It contains eight learned layers —
five convolutional and three fully-connected.

ryefoxlime/PneumoniaDetection
Resnet
Finetuned Paper - https://pubmed.ncbi.nlm.nih.gov/32501424/
Already trained on Covid-19, pneumonia and normal classes


## Helpful Articles about CNNs and ImageNets
https://towardsdatascience.com/the-w3h-of-alexnet-vggnet-resnet-and-inception-7baaaecccc96#:~:text=AlexNet%20was%20born%20out%20of,with%20an%20accuracy%20of%2073.8%25.

